In [55]:
import pandas as pd
import numpy as np
import re
import difflib
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import string
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import GaussianNB
from sklearn import model_selection, naive_bayes, svm
from nltk.collocations import BigramCollocationFinder, BigramAssocMeasures
import warnings
warnings.filterwarnings("ignore")
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
import nltk
import logging
from keras.models import Sequential
from keras import layers
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from keras.preprocessing.sequence import pad_sequences

In [39]:
data = pd.read_csv('Full+data.csv')
data.head(1)

,product_id,brand,mpn,name,description,brand_category,created_at,updated_at,deleted_at,brand_canonical_url,...,Unnamed: 246,Unnamed: 247,Unnamed: 248,Unnamed: 249,Unnamed: 250,Unnamed: 251,Unnamed: 252,Unnamed: 253,Unnamed: 254,Unnamed: 255
0,01DSRPSZTDW2PGK1YWYXJGKZZ0,FILA,400010319073,Original Fitness Sneakers,Vintage Fitness leather sneakers with logo pri...,TheMensStore/Shoes/Sneakers/LowTop,2019-11-15 23:36:38.98161+00,2019-12-19 20:40:30.786144+00,NaN,https://www.saksfifthavenue.com/fila-original-...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [40]:
# reduce to non nan columns
cols = data.columns[:14]
data = data.loc[:,cols]
data.shape

(42373, 14)

In [41]:
# import extra data
extra_data = pd.read_csv('extra_data.csv')
extra_data.head(1)


,product_id,mpn,brand,brand_category,name,labels,updated_at,bc_product_id,saleable,details,created_at,brand_canonical_url,notes,description
0,01E5ZXP5H0BTEZT9QD2HRZJ47A,5529544,A.L.C.,Unknown,Lennox High Waist Cotton & Linen Pants,[],2020-04-17 15:44:57.785000+00:00,5021,2020-04-17 15:44:57.785000+00:00,"True to size. High rise.\n31"" inseam; 14"" leg ...",2020-04-15 21:59:56.695000+00:00,https://shop.nordstrom.com/s/a-l-c-lennox-high...,NaN,High-rise trousers tailored from a cool Italia...


In [42]:
# columns to use from extra_data
cols = ['product_id', 'brand', 'name', 'description', 'brand_category', 'brand_canonical_url','details']

In [43]:
extra_data = extra_data.loc[:,cols]
data = data.loc[:,cols]

In [44]:
# concatenate data and extra data
full_data = pd.concat([data,extra_data])
len(full_data) == len(data) + len(extra_data)

True

In [45]:
full_data.fillna('UNKNOWNTOKEN',inplace=True)

In [46]:
# we will join brand and brand_category together to brand_info for use in predictions

#full_data['brand_info'] = full_data['brand']+full_data['brand_category']+ " " +full_data['brand_canonical_url']

In [47]:
full_data.isnull().sum()

product_id             0
brand                  0
name                   0
description            0
brand_category         0
brand_canonical_url    0
details                0
brand_info             0
dtype: int64

In [60]:
full_data = full_data.reset_index(drop = True)

In [56]:
# function to remove stopwords
from gensim.parsing.preprocessing import STOPWORDS
stop = set(STOPWORDS)
def remove_stopwords(data_col):
    new_list = []
    a = data_col
    for i in range(0,len(a)):
        words = word_tokenize(a[i])
        res_words = []
        for word in words:
            if word not in stop:
                res_words.append(word)
            sentence = " ".join(res_words)
        new_list.append(sentence)
    return new_list

In [61]:
# remove stopwords from each of the 4 columns
full_data['description'] = remove_stopwords(full_data['description'])
full_data['details'] = remove_stopwords(full_data['details'])
full_data['name'] = remove_stopwords(full_data['name'])
full_data['brand'] = remove_stopwords(full_data['brand'])
full_data['brand_category'] = remove_stopwords(full_data['brand_category'])

In [62]:
# function to do regex cleaning
def reg_clean(data,col):
    new_list = []
    for i in range(0,len(data)):
        #special characters 
        a = re.sub(r'[^ a-zA-Z0-9]',' ',data.loc[i,col])
        #new lines
        a = re.sub(r'\n',' ', a)
        #remove multiple spaces by a single space
        a = re.sub(r'\s+',' ',a)
        #timestamp
        a = re.sub(r'\b[0-9]{1,}am|[0-9]{1,}pm|[0-9]{4,}|[0-9]ish|1st|2nd|3rd|[0-9]{1,2}th|31st|[0-9]{1,}min(?:utes)?s?|[0-9]{1,}h(?:ou)?rs?|[0-9]{3,}\b','timestamp',a)
        a = re.sub(r'\b[0-9]{1,}timestamp\b','timestamp',a)
        #any numbers as digit
        a = re.sub(r'\b\d+\b','digit',a)
        #number followed by a variable
        a = re.sub(r'\b\d{1,}[a-z]{0,}[0-9]{0,}','varchar',a)
        #html codes
        a = re.sub(r'<.+?>','html',a)
        a = re.sub(r'https|www|com','html',a)
        # additional stopwords
        a = re.sub(r'\ba\b', '', a, flags = re.IGNORECASE)
        a = re.sub(r'\bthis|the\b', '',a, flags = re.IGNORECASE)
        new_list.append(a)
    return new_list

In [63]:
full_data['description'] = remove_stopwords(full_data['description'])
full_data['details'] = remove_stopwords(full_data['details'])
full_data['name'] = remove_stopwords(full_data['name'])
full_data['brand'] = remove_stopwords(full_data['brand'])
full_data['brand_category'] = remove_stopwords(full_data['brand_category'])

In [64]:
# function to do regex cleaning
def reg_clean(data,col):
    new_list = []
    for i in range(0,len(data)):
        #special characters 
        a = re.sub(r'[^ a-zA-Z0-9]',' ',data.loc[i,col])
        #new lines
        a = re.sub(r'\n',' ', a)
        #remove multiple spaces by a single space
        a = re.sub(r'\s+',' ',a)
        #timestamp
        a = re.sub(r'\b[0-9]{1,}am|[0-9]{1,}pm|[0-9]{4,}|[0-9]ish|1st|2nd|3rd|[0-9]{1,2}th|31st|[0-9]{1,}min(?:utes)?s?|[0-9]{1,}h(?:ou)?rs?|[0-9]{3,}\b','timestamp',a)
        a = re.sub(r'\b[0-9]{1,}timestamp\b','timestamp',a)
        #any numbers as digit
        a = re.sub(r'\b\d+\b','digit',a)
        #number followed by a variable
        a = re.sub(r'\b\d{1,}[a-z]{0,}[0-9]{0,}','varchar',a)
        #html codes
        a = re.sub(r'<.+?>','html',a)
        a = re.sub(r'https|www|com','html',a)
        # additional stopwords
        a = re.sub(r'\ba\b', '', a, flags = re.IGNORECASE)
        a = re.sub(r'\bthis|the\b', '',a, flags = re.IGNORECASE)
        new_list.append(a)
    return new_list

In [65]:
# perform regex cleaning on each column
full_data['description'] = reg_clean(full_data,'description')
full_data['details'] = reg_clean(full_data,'details')
full_data['brand'] = reg_clean(full_data,'brand')
full_data['name'] = reg_clean(full_data, 'name')
full_data['brand_category'] = reg_clean(full_data,'brand_category')

In [66]:
# function to lemmatize columns
def lemmatize_sentence(data_col):
    new_list = []
    a = data_col 
    for i in range(0,len(a)):
        words = word_tokenize(a[i])
        res_words = []
        for word in words:
            res_words.append(lemmatizer.lemmatize(word).strip(string.punctuation))
        sentence = " ".join(res_words)
        new_list.append(sentence)
    return new_list

In [67]:
# lemmatize each column
full_data['description'] = lemmatize_sentence(full_data['description'])
full_data['details'] = lemmatize_sentence(full_data['details'])
full_data['brand'] = lemmatize_sentence(full_data['brand'])
full_data['brand_category'] = lemmatize_sentence(full_data['brand_category'])
full_data['name'] = lemmatize_sentence(full_data['name'])

In [68]:
# function to start collocation
stopwords_coll = set(stopwords.words('english') + [".",'.', ",",":", "''", "'s", "'", "``", "(", ")", "-","timestamp","varchar","html","digit"])
filter_stops = lambda w: len(w) < 3 or w in stopwords_coll
def collocation_list(data_col):
    new_list = []
    for i in range(0,len(data_col)):
        words = word_tokenize(data_col[i])
        res_words = []
        for word in words:
            if word not in stopwords_coll:
                res_words.append(word)
        new_list.append(res_words)
    return(new_list)

In [69]:
cols = ['brand', 'brand_category', 'description', 'name','details']
n_list = []
for col in cols:
    n_list.append(collocation_list(full_data[col]))

In [70]:
# we find that dry clean, machine wash, 'fits true to size', 'made in italy' 
# should be collated together

col_list = [['Dry', 'clean'], ['Machine', 'wash'], ['true', 'size'],['fit', 'true_size'], ['Fits', 'true_size'],  
            ['Made', 'Italy']]
data_col = full_data['details']
new_list = []
clean_list = ['Dry','wash','Italy', 'size']
for i in range(0,len(data_col)):
    words = word_tokenize(data_col[i])
    len_words = len(words)-1
    for i in range(0,len_words):
        bi_word = []
        j = i+1
        bi_word.append(words[i])
        bi_word.append(words[j])
        if(bi_word in col_list):
            sentence = "_".join(bi_word)
            words[i] = sentence
    for word in words:
        if word in clean_list:
            words.remove(word)
    res_word = []
    for word in words:
        res_word.append(word)
    sentence = " ".join(res_word)
    new_list.append(sentence)
full_data['details'] = new_list

In [71]:
# we find that wide leg, ankle boot, high waist, high rise 
# should be collated together

col_list = [['wide','leg'], ['ankle', 'boot'], ['High', 'Waist'], ['Wide', 'Leg'], ['High', 'Rise'], 
            ['straight', 'leg'], ['shoulder', 'bag']]
data_col = full_data['name']
new_list = []
clean_list = ['leg', 'boot', 'Waist', 'Leg', 'Rise','bag']
for i in range(0,len(data_col)):
    words = word_tokenize(data_col[i])
    len_words = len(words)-1
    for i in range(0,len_words):
        bi_word = []
        j = i+1
        bi_word.append(words[i])
        bi_word.append(words[j])
        if(bi_word in col_list):
            sentence = "_".join(bi_word)
            words[i] = sentence
    for word in words:
        if word in clean_list:
            words.remove(word)
    res_word = []
    for word in words:
        res_word.append(word)
    sentence = " ".join(res_word)
    new_list.append(sentence)
full_data['name'] = new_list

In [72]:
col_list = [['Dry','clean'], ['Machine', 'wash'], ['Made', 'Italy'], ['wide', 'leg'], ['best', 'selling'], 
            ['high', 'waisted'], ['pencil', 'skirt'], ['High', 'Rise'], ['high','rise'], ['Hand','wash']]
data_col = full_data['description']
new_list = []
clean_list = ['clean', 'wash', 'Italy', 'leg', 'selling','waisted', 'skirt','Rise', 'rise']
for i in range(0,len(data_col)):
    words = word_tokenize(data_col[i])
    len_words = len(words)-1
    for i in range(0,len_words):
        bi_word = []
        j = i+1
        bi_word.append(words[i])
        bi_word.append(words[j])
        if(bi_word in col_list):
            sentence = "_".join(bi_word)
            words[i] = sentence
    for word in words:
        if word in clean_list:
            words.remove(word)
    res_word = []
    for word in words:
        res_word.append(word)
    sentence = " ".join(res_word)
    new_list.append(sentence)
full_data['description'] = new_list

In [73]:
# we find that  should be collocated together

col_list = [['New', 'York'], ['Jimmy','Choo'], ['Banana','Republic'], ['Victoria','Beckham']]
data_col = data_color['brand']
new_list = []
clean_list = ['York', 'Choo','Republic', 'Beckham']
for i in range(0,len(data_col)):
    words = word_tokenize(data_col[i])
    len_words = len(words)-1
    for i in range(0,len_words):
        bi_word = []
        j = i+1
        bi_word.append(words[i])
        bi_word.append(words[j])
        if(bi_word in col_list):
            sentence = "_".join(bi_word)
            words[i] = sentence
    for word in words:
        if word in clean_list:
            words.remove(word)
    res_word = []
    for word in words:
        res_word.append(word)
    sentence = " ".join(res_word)
    new_list.append(sentence)
data_color['brand'] = new_list

In [ ]:
col_list = [['Mid', 'Heel'], ['Shoulder', 'Bags']]
data_col = data_color['brand_category']
new_list = []
clean_list = ['Heel', 'Bags']
for i in range(0,len(data_col)):
    words = word_tokenize(data_col[i])
    len_words = len(words)-1
    for i in range(0,len_words):
        bi_word = []
        j = i+1
        bi_word.append(words[i])
        bi_word.append(words[j])
        if(bi_word in col_list):
            sentence = "_".join(bi_word)
            words[i] = sentence
    for word in words:
        if word in clean_list:
            words.remove(word)
    res_word = []
    for word in words:
        res_word.append(word)
    sentence = " ".join(res_word)
    new_list.append(sentence)
data_color['brand_category'] = new_list

In [74]:
# initialize the tf-idf vectorizer
vectorizer = TfidfVectorizer(token_pattern=r'\b[a-zA-Z]{3,}\b',stop_words="english",binary = True,min_df = 0.005,max_df = 0.7,max_features =300)

In [75]:
# vectorizing our test/full data
columns = ['description','details','brand', 'brand_category', 'name']
model_data_test=pd.DataFrame()
for j in columns:
    corpus = []
    for i in range(0,len(full_data)):
        corpus.append(full_data.loc[i,j])
    vect = vectorizer.fit_transform(corpus)
    terms = vectorizer.get_feature_names()
    c=pd.DataFrame(vect.toarray().transpose(), index=terms)
    model_data_test=pd.concat([model_data_test,c.T],axis = 1)

1. have your pre-made df
2. Train model on pre-made df
2. make a df with just the 4 input columns
3. have prof input as last row of df
4. run all pre-processing, vectorization, etc on the df
5. when you do train/test split, train = everything but input, test = input
6. convert test output to string


In [76]:
training_data = pd.read_csv('subset_color.csv')

In [80]:
# vectorize training data
columns = ['description','details','brand', 'brand_category', 'name']
model_data_train=pd.DataFrame()
for j in columns:
    corpus = []
    for i in range(0,len(training_data)):
        corpus.append(training_data.loc[i,j])
    vect = vectorizer.fit_transform(corpus)
    terms = vectorizer.get_feature_names()
    c=pd.DataFrame(vect.toarray().transpose(), index=terms)
    model_data_train=pd.concat([model_data_train,c.T],axis = 1)

In [81]:
from sklearn.linear_model import LogisticRegression

In [89]:
model_data_train.shape

(39942, 1076)

In [90]:
model_data_test.shape

(48994, 1035)

In [91]:
#Train model for black
X=model_data_train
y=training_data['is_black'].values
print(training_data['is_black'].sum()/len(training_data))
logreg_black=LogisticRegression(n_jobs=1, C=1e5)      
logreg_black.fit(X, y)
y_pred = logreg_black.predict(model_data_test)
print('accuracy %s' % accuracy_score(y_pred, y_test))

0.4881578288518352


ValueError: X has 1035 features per sample; expecting 1076

In [84]:
color_class = ['is_black', 'is_beige', 'is_burgundy', 'is_white', 'is_gray', 'is_gold', 'is_blue', 'is_neutral', 
               'is_pink', 'is_orange', 'is_navy', 'is_brown', 'is_red', 'is_yellow', 'is_multi', 'is_green', 
               'is_silver', 'is_teal', 'is_purple']

In [85]:
tag_df = pd.DataFrame(columns = color_class)

In [88]:
# have our model predict our entire test data
y_pred_black = logreg_black.predict(model_data_test)

ValueError: X has 1035 features per sample; expecting 1076

Plan is to do the above code and pass them all into a dataframe, then convert data frame to color name string then append string as column onto original full_data set
would need to attach via product id